# Deterministic Thinking and Its Limits: When Models Assume Certainty

This notebook demonstrates a fundamental limit of optimization: **deterministic thinking** assumes inputs are fixed and certain, but the real world is uncertain.

Understanding this limit is critical because:
- Optimization models assume inputs are known exactly
- Reality is uncertain: demand fluctuates, costs change, conditions vary
- When inputs differ from assumptions, optimal solutions become suboptimal
- Results are best estimates, not guarantees


## Key Concepts

**Deterministic Models** assume inputs are fixed and certain:
- Demand forecasts are treated as exact values
- Cost estimates are treated as certain
- Conditions are assumed to be stable
- The model optimizes for these specific values

**Reality is Uncertain**:
- Demand fluctuates (higher or lower than forecasted)
- Costs vary (increase or decrease)
- Conditions change (breakdowns, maintenance, disruptions)

**Critical insight**: Models assume certainty; reality is uncertain. When reality differs from assumptions, optimal solutions become suboptimal. Results are best estimates, not guarantees.


## Scenario: Production Planning with Demand Uncertainty

You need to schedule production based on a demand forecast. But actual demand may differ from the forecast.

**Decision**: How many units to produce?

**Model assumption**: Demand will be exactly as forecasted (deterministic)

**Reality**: Demand is uncertain and may vary


## Step 1: Install Required Packages (Colab)

If you're running this notebook in Google Colab, you need to install the `pulp` package first. This cell can be skipped if running locally and the package is already installed.


In [ ]:
# Install pulp package (required for optimization)
# This is needed in Google Colab; can be skipped if already installed locally
%pip install pulp -q


## Step 2: Import Libraries


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value


## Step 3: Optimize with Forecasted Demand (Deterministic)

The model assumes demand will be exactly as forecasted:


In [ ]:
# Problem inputs
profit_A = 50  # Profit per unit of Product A ($)
profit_B = 40  # Profit per unit of Product B ($)

capacity = 2000  # Total production capacity (units)
labor_hours = 1500  # Available labor hours
labor_per_A = 0.5  # Labor hours per unit of A
labor_per_B = 0.8  # Labor hours per unit of B

# FORECASTED demand (what the model assumes)
forecast_demand_A = 1000  # Forecast: 1,000 units
forecast_demand_B = 800   # Forecast: 800 units

print("MODEL ASSUMPTIONS (Deterministic):")
print("=" * 60)
print(f"  Forecasted Demand A: {forecast_demand_A} units (assumed exact)")
print(f"  Forecasted Demand B: {forecast_demand_B} units (assumed exact)")
print(f"  The model treats these as CERTAIN values")

# Create optimization model
model = LpProblem("Production_Deterministic", LpMaximize)

# Decision variables
produce_A = LpVariable("produce_A", lowBound=0, cat='Continuous')
produce_B = LpVariable("produce_B", lowBound=0, cat='Continuous')

# Objective: Maximize profit
model += profit_A * produce_A + profit_B * produce_B, "Total_Profit"

# Constraints
model += produce_A + produce_B <= capacity, "Capacity_Limit"
model += labor_per_A * produce_A + labor_per_B * produce_B <= labor_hours, "Labor_Limit"
model += produce_A <= forecast_demand_A, "Demand_A_Limit"  # Uses forecast (assumed exact)
model += produce_B <= forecast_demand_B, "Demand_B_Limit"  # Uses forecast (assumed exact)

# Solve
model.solve()

optimal_A_forecast = value(produce_A)
optimal_B_forecast = value(produce_B)
optimal_profit_forecast = value(model.objective)

print(f"\nOPTIMAL SOLUTION (Based on Forecast):")
print(f"  Produce {optimal_A_forecast:.1f} units of Product A")
print(f"  Produce {optimal_B_forecast:.1f} units of Product B")
print(f"  Expected Profit: ${optimal_profit_forecast:,.2f}")
print(f"\nThis solution is optimal FOR THE FORECASTED DEMAND.")
print(f"The model assumes demand will be exactly as forecasted.")


## Step 4: What If Actual Demand Differs? (Reality Is Uncertain)

Now let's see what happens when actual demand is different from the forecast:


In [ ]:
# Scenario 1: Actual demand is HIGHER than forecast
actual_demand_A_high = 1200  # Actual: 1,200 units (20% higher than forecast)
actual_demand_B_high = 900   # Actual: 900 units (12.5% higher than forecast)

print("SCENARIO 1: Actual Demand is HIGHER than Forecast")
print("=" * 60)
print(f"  Forecasted: A={forecast_demand_A}, B={forecast_demand_B}")
print(f"  Actual:     A={actual_demand_A_high}, B={actual_demand_B_high}")
print(f"  Difference: A=+{actual_demand_A_high - forecast_demand_A} (+{((actual_demand_A_high - forecast_demand_A)/forecast_demand_A)*100:.1f}%)")
print(f"              B=+{actual_demand_B_high - forecast_demand_B} (+{((actual_demand_B_high - forecast_demand_B)/forecast_demand_B)*100:.1f}%)")

# Evaluate the "optimal" solution with actual demand
# The solution was optimal for forecast, but what about actual demand?
actual_profit_high = profit_A * min(optimal_A_forecast, actual_demand_A_high) + \
                     profit_B * min(optimal_B_forecast, actual_demand_B_high)

# What would be optimal for actual demand?
model_actual_high = LpProblem("Production_Actual_High", LpMaximize)
produce_A_high = LpVariable("produce_A_high", lowBound=0, cat='Continuous')
produce_B_high = LpVariable("produce_B_high", lowBound=0, cat='Continuous')

model_actual_high += profit_A * produce_A_high + profit_B * produce_B_high, "Total_Profit"
model_actual_high += produce_A_high + produce_B_high <= capacity, "Capacity_Limit"
model_actual_high += labor_per_A * produce_A_high + labor_per_B * produce_B_high <= labor_hours, "Labor_Limit"
model_actual_high += produce_A_high <= actual_demand_A_high, "Demand_A_Limit"
model_actual_high += produce_B_high <= actual_demand_B_high, "Demand_B_Limit"

model_actual_high.solve()
optimal_A_actual_high = value(produce_A_high)
optimal_B_actual_high = value(produce_B_high)
optimal_profit_actual_high = value(model_actual_high.objective)

print(f"\nCOMPARISON:")
print(f"  Solution based on forecast: ${optimal_profit_forecast:,.2f}")
print(f"  Actual profit with forecast solution: ${actual_profit_high:,.2f}")
print(f"  Optimal solution for actual demand: ${optimal_profit_actual_high:,.2f}")
print(f"\n  Opportunity lost: ${optimal_profit_actual_high - actual_profit_high:,.2f}")
print(f"\nKey Insight:")
print(f"  - The solution optimal for forecast is NOT optimal for actual demand")
print(f"  - The model assumed certainty, but reality was different")


In [ ]:
# Scenario 2: Actual demand is LOWER than forecast
actual_demand_A_low = 800   # Actual: 800 units (20% lower than forecast)
actual_demand_B_low = 700   # Actual: 700 units (12.5% lower than forecast)

print("SCENARIO 2: Actual Demand is LOWER than Forecast")
print("=" * 60)
print(f"  Forecasted: A={forecast_demand_A}, B={forecast_demand_B}")
print(f"  Actual:     A={actual_demand_A_low}, B={actual_demand_B_low}")
print(f"  Difference: A={actual_demand_A_low - forecast_demand_A} ({((actual_demand_A_low - forecast_demand_A)/forecast_demand_A)*100:.1f}%)")
print(f"              B={actual_demand_B_low - forecast_demand_B} ({((actual_demand_B_low - forecast_demand_B)/forecast_demand_B)*100:.1f}%)")

# Evaluate the "optimal" solution with actual demand
actual_profit_low = profit_A * min(optimal_A_forecast, actual_demand_A_low) + \
                    profit_B * min(optimal_B_forecast, actual_demand_B_low)

# What would be optimal for actual demand?
model_actual_low = LpProblem("Production_Actual_Low", LpMaximize)
produce_A_low = LpVariable("produce_A_low", lowBound=0, cat='Continuous')
produce_B_low = LpVariable("produce_B_low", lowBound=0, cat='Continuous')

model_actual_low += profit_A * produce_A_low + profit_B * produce_B_low, "Total_Profit"
model_actual_low += produce_A_low + produce_B_low <= capacity, "Capacity_Limit"
model_actual_low += labor_per_A * produce_A_low + labor_per_B * produce_B_low <= labor_hours, "Labor_Limit"
model_actual_low += produce_A_low <= actual_demand_A_low, "Demand_A_Limit"
model_actual_low += produce_B_low <= actual_demand_B_low, "Demand_B_Limit"

model_actual_low.solve()
optimal_A_actual_low = value(produce_A_low)
optimal_B_actual_low = value(produce_B_low)
optimal_profit_actual_low = value(model_actual_low.objective)

print(f"\nCOMPARISON:")
print(f"  Solution based on forecast: ${optimal_profit_forecast:,.2f}")
print(f"  Actual profit with forecast solution: ${actual_profit_low:,.2f}")
print(f"  Optimal solution for actual demand: ${optimal_profit_actual_low:,.2f}")
print(f"\n  Opportunity lost: ${optimal_profit_actual_low - actual_profit_low:,.2f}")
print(f"\nKey Insight:")
print(f"  - When demand is lower, the forecast-based solution overproduces")
print(f"  - This creates waste or excess inventory")
print(f"  - The model assumed certainty, but reality was different")


## Step 6: Sensitivity Analysis: How Sensitive Is the Solution?

Let's see how the optimal solution changes as demand varies:


In [ ]:
# Sensitivity analysis: Vary demand and see how optimal solution changes
demand_variations = np.arange(0.7, 1.4, 0.1)  # 70% to 130% of forecast

sensitivity_results = []

for multiplier in demand_variations:
    test_demand_A = int(forecast_demand_A * multiplier)
    test_demand_B = int(forecast_demand_B * multiplier)
    
    # Optimize for this demand level
    model_test = LpProblem("Production_Test", LpMaximize)
    produce_A_test = LpVariable("produce_A_test", lowBound=0, cat='Continuous')
    produce_B_test = LpVariable("produce_B_test", lowBound=0, cat='Continuous')
    
    model_test += profit_A * produce_A_test + profit_B * produce_B_test, "Total_Profit"
    model_test += produce_A_test + produce_B_test <= capacity, "Capacity_Limit"
    model_test += labor_per_A * produce_A_test + labor_per_B * produce_B_test <= labor_hours, "Labor_Limit"
    model_test += produce_A_test <= test_demand_A, "Demand_A_Limit"
    model_test += produce_B_test <= test_demand_B, "Demand_B_Limit"
    
    model_test.solve()
    
    optimal_A_test = value(produce_A_test)
    optimal_B_test = value(produce_B_test)
    optimal_profit_test = value(model_test.objective)
    
    sensitivity_results.append({
        'Demand Multiplier': f"{multiplier:.1f}x",
        'Demand A': test_demand_A,
        'Demand B': test_demand_B,
        'Optimal A': optimal_A_test,
        'Optimal B': optimal_B_test,
        'Optimal Profit': optimal_profit_test
    })

sensitivity_df = pd.DataFrame(sensitivity_results)
print("SENSITIVITY ANALYSIS: How Optimal Solution Changes with Demand")
print("=" * 80)
display(sensitivity_df.round(2))

print("\nKey Insight:")
print("  - Optimal solution changes as demand changes")
print("  - The model assumes fixed demand, but reality varies")
print("  - This shows why results are estimates, not guarantees")


## Step 7: Visualize Sensitivity

Let's visualize how the optimal solution and profit change with demand:


In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 10))

# Plot 1: Optimal production quantities
ax1 = axes[0]
multipliers = [float(x.replace('x', '')) for x in sensitivity_df['Demand Multiplier']]
ax1.plot(multipliers, sensitivity_df['Optimal A'], 'o-', label='Product A', linewidth=2, markersize=8)
ax1.plot(multipliers, sensitivity_df['Optimal B'], 's-', label='Product B', linewidth=2, markersize=8)
ax1.axvline(x=1.0, color='red', linestyle='--', linewidth=2, label='Forecast (1.0x)')
ax1.set_xlabel('Demand Multiplier (relative to forecast)', fontsize=12)
ax1.set_ylabel('Optimal Production Quantity', fontsize=12)
ax1.set_title('Optimal Solution Changes with Demand\n(Deterministic Model Assumes Fixed Demand)', fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Plot 2: Optimal profit
ax2 = axes[1]
ax2.plot(multipliers, sensitivity_df['Optimal Profit'], 'o-', color='green', linewidth=2, markersize=8)
ax2.axvline(x=1.0, color='red', linestyle='--', linewidth=2, label='Forecast (1.0x)')
ax2.set_xlabel('Demand Multiplier (relative to forecast)', fontsize=12)
ax2.set_ylabel('Optimal Profit ($)', fontsize=12)
ax2.set_title('Optimal Profit Changes with Demand\n(Reality: Demand Varies)', fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Insight:")
print("  - The optimal solution is sensitive to demand")
print("  - Models assume fixed inputs, but reality varies")
print("  - This is why optimization results are estimates, not guarantees")


## Summary: Deterministic Thinking and Its Limits

**Deterministic Models**:
- Assume inputs are fixed and certain
- Treat forecasts as exact values
- Optimize for specific assumed values
- Do not account for uncertainty

**Reality is Uncertain**:
- Demand fluctuates
- Costs vary
- Conditions change
- Inputs differ from assumptions

**What This Means**:
- Optimization finds the best solution for assumed inputs
- If inputs are wrong, the solution is not optimal for reality
- Results are best estimates, not guarantees
- Monitoring and adjustment are essential

**Critical insight**: 
- Models assume certainty; reality is uncertain
- This creates a fundamental limit
- Understanding this limit helps you use optimization correctly

**Practical implication**:
- Treat optimization results as estimates, not guarantees
- Monitor actual conditions vs assumed conditions
- Adjust solutions as conditions change
- Recognize that optimization assumes certainty in an uncertain world
